# 1

In [1]:
%1
T = dlmread('Q1/RESOURCES/NGS_GRAVD_Block_CN03_BETA1/NGS_GRAVD_Block_CN03_Gravity_Data_BETA1.txt');
place = T(:,1);
time = T(:,2);
latitude = T(:,3);
longitude = T(:,4);
elipsoidal_height = T(:,5);
gravity_filtered = T(:,6);

# 2

In [30]:
%2 
%N_ggm = open('/Users/sunilbhavsar/Documents/CE678_PROJECT/Q2/RESOURCES/AIUB-CHAMP01S_a50de3dfc2ab0cdb9a2ae28b113f3398304202075281a813ba9cc27562a667af.gdf')
%a = open('/Users/sunilbhavsar/Documents/CE678_PROJECT/AIUB-CHAMP01S_a50de3dfc2ab0cdb9a2ae28b113f3398304202075281a813ba9cc27562a667af.gdf')
fid = fopen('Q2/Downloaded_data/GGM01C.gdf'); % file identifier

h = true; % while loop condition

n = 1; % headerlines counter

while true

    str = fgets(fid); % reading the file line by line
    
    if length(str) >= 11
        indx = strcmp(str(1:11), 'end_of_head'); % checking for 'end_of_head' string
    else
        indx = false;
    end
    
    if indx == 0
    
    n = n+1; % keep counting
    
    else
    
    h = false; % break loop
    break;
    
    end

end
str = fgets(fid);
fclose(fid);
GGM01C = textread('Q2/Downloaded_data/GGM01C.gdf','', 'headerlines', n);

In [31]:
GGM01C_longitudes = GGM01C(:,1)-360;
GGM01C_latitudes = GGM01C(:,2);
GGM01C_geoid = GGM01C(:,3);